# Fantasy Football Prep

The League I'm in has the following parameters:
1. 1 QB
2. 2 RB
3. 3 WR
4. 1 TE 
5. 1 flex
6. 1 D/ST
7. 1 K

The goal of this notebook is to provide draft-day assistance by interfacing with fantasy pro's aggregation of rankings and draft statistics.

In [38]:

def check_roster(open_roster,rules):
    
    
    # do a uniqueness check?
    
    pass

In [43]:
print np.unique(roster)
#print roster.remove('QB')
#print roster
#print roster.remove('QB')

['DST' 'K' 'QB']


In [2]:
# boilerplate imports

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import matplotlib as mpl
cmap = cm.gnuplot





In [3]:
# imports for scraping

from bs4 import BeautifulSoup

import requests

import operator




In [114]:

def find_best_available(position,start_point=0,des_position='QB'):
    
    indx = start_point
    filled = 0
    
    
    while (indx < len(listing)) & (filled == 0):
        
        if position[indx] == des_position:
            
            filled = 1
            
            return indx
            
        indx += 1
        

    if filled == 0:
        
        print 'out back'
        return -1

    
def calculate_snake(pos,rounds=12,teams=12):
    double = [pos,float(teams)+1.-pos]
    order = []
    for rnd in range(0,rounds,2):
        order.extend(double)
        
    return float(teams)*(np.arange(0.,float(rounds),1.)) + np.array(order)



def extract_position(pos):
    
    if sav[2][0] == 'K':
        return 'K'
    
    elif sav[2][0] == 'D':
                    
        return 'DST'
                
    elif sav[2][0] =='W':

        return 'WR'

    else:
        return sav[2][0:2]


    
    


## Basic Application: Kicker Rankings

In [116]:
r  = requests.get('https://www.fantasypros.com/nfl/rankings/half-point-ppr-cheatsheets.php')

# standard league
#https://www.fantasypros.com/nfl/rankings/consensus-cheatsheets.php

# [u'Rank\r\n\r\n', u'Player (Team)', u'Pos', u'Bye', \
#         0              1              2       3
#     u'Best', u'Worst', u'Avg', u'Std Dev', u'ADP', u'vs. ADP', u'Notes']
#        4          5      6        7           8         9        10

data = r.text; soup = BeautifulSoup(data); tables = soup.findAll('table')


knum = 1

for indx,table in enumerate(tables): 
       
    if indx==0:
                
        for row in table.find_all("tr")[2:]:
            sav = [td.get_text() for td in row.find_all("td")]
            
            try:
                if sav[2][0] == 'K':
                    #print sav[2][0:1]
                    
                    if knum < 13:
                        print knum,sav[0],sav[1]
                        
                    knum += 1
                elif sav[2][0] == 'D':
                    
                    #print sav[2][0:3]
                    pass
                else:
                    #print sav[2][0:2]
                    pass
                
            except:
                pass
        


1 170 Justin Tucker BAL 
2 173 Stephen Gostkowski NE 
3 192 Dan Bailey DAL 
4 195 Matt Bryant ATL 
5 199 Mason Crosby GB 
6 218 Adam Vinatieri IND 
7 230 Steven Hauschka BUF 
8 245 Matt Prater DET 
9 247 Cairo Santos KC 
10 252 Sebastian Janikowski OAK 
11 257 Will Lutz NO 
12 258 Brandon McManus DEN 


These are the Kickers worth drafting in the latest rounds, do not draft anyone else!

In [130]:
r  = requests.get('https://www.fantasypros.com/nfl/rankings/half-point-ppr-cheatsheets.php')

# standard league
#https://www.fantasypros.com/nfl/rankings/consensus-cheatsheets.php

# [u'Rank\r\n\r\n', u'Player (Team)', u'Pos', u'Bye', \
#         0              1              2       3
#     u'Best', u'Worst', u'Avg', u'Std Dev', u'ADP', u'vs. ADP', u'Notes']
#        4          5      6        7           8         9        10


data = r.text

soup = BeautifulSoup(data)

tables = soup.findAll('table')

ordering = calculate_snake(4.,rounds=16)

ordinal = 6 # 6 in ranking
ordinal = 8 # 8 is ADP

listing = []
player = []
position = []
notes = []
for indx,table in enumerate(tables): 
    
    #print indx,[th.get_text() for th in table.find("tr").find_all("th")]
   
    if indx==0:
        
        #print [th.get_text() for th in table.find("tr").find_all("th")]
        
        for row in table.find_all("tr")[1:]:
            sav = [td.get_text() for td in row.find_all("td")]
            # get the links here and then think about scraping those pages
            
            #try:
            #    sav2 = [td.find('a') for td in row.find_all("td")][1]
            #    print 'https://www.fantasypros.com'+sav2.get('href')
            #except:
            #    pass
            
            try:
                #if float(sav[0]) in ordering:
                #    print sav[0],sav[1],sav[2]#,sav[10]
                listing.append(float(sav[ordinal]))
                player.append(sav[1])
                position.append(extract_position(sav[2]))
                notes.append(sav[10])
                
            except:
                pass#print 'fail'
                
            
NU = np.array(listing)
PL = np.array(player)
PO = np.array(position)
NO = np.array(notes)
# organize as specified
reorg = NU.argsort()

#print PO


#https://www.fantasypros.com/nfl/players/david-johnson-rb.php

# replace 'players' with 'stats'

In [131]:

# what should the flex position be filled with?
flex = 'WR'
bench1 = 'WR'
bench2 = 'WR'
bench3 = 'QB'
bench4 = 'RB'
bench5 = 'WR'
bench6 = 'RB'

roster = ['QB','RB','RB','WR','WR','WR','TE',flex,'DST','K',bench1,bench2,bench3,bench4,bench5,bench6]



                    
for entry,value in enumerate(NU[reorg]):
    
    if (float(entry) + 1.) in ordering:
        
        #print roster
        
        if PO[reorg[entry]] in roster:
        
            print('{0:<6.1f}{1:<24s}{2:>5s}'.format(NU[reorg[entry]],PL[reorg[entry]],PO[reorg[entry]]))
            
            roster.remove(PO[reorg[entry]])
            
            print ''
            print NO[reorg[entry]]
            print ''
            
        else:
            
            #print 'Moving to next draft slot...'
            # try to draft down the line to next position
            left_to_draft = np.unique(roster)
            
            next_bests = np.array([find_best_available(PO[reorg],entry,rospos) for rospos in left_to_draft])
            
            increase = np.min(next_bests)
            print('{0:<6.1f}{1:<24s}{2:>5s}'.format(NU[reorg[int(increase)]],PL[reorg[int(increase)]],PO[reorg[int(increase)]]))
            roster.remove(PO[reorg[int(increase)]])

            print ''
            print NO[reorg[increase]]
            print ''
            
        #print ''
        
        #print 'Next Best QB (+%i):' %(find_best_available(PO,entry+2,'QB')-(entry+2)),PL[find_best_available(PO,entry+1,'QB')]
        #print 'Next Best WR (+%i):' %(find_best_available(PO,entry+2,'WR')-(entry+2)),PL[find_best_available(PO,entry+1,'WR')]
        #print 'Next Best RB (+%i):' %(find_best_available(PO,entry+2,'RB')-(entry+2)),PL[find_best_available(PO,entry+1,'RB')]
        #print 'Next Best TE (+%i):' %(find_best_available(PO,entry+2,'TE')-(entry+2)),PL[find_best_available(PO,entry+1,'TE')]
        #print 'Next Best DST (+%i):' %(find_best_available(PO,entry+2,'DST')-(entry+2)),PL[find_best_available(PO,entry+1,'DST')]
        #print 'Next Best K (+%i):' %(find_best_available(PO,entry+2,'K')-(entry+2)),PL[find_best_available(PO,entry+1,'K')]
        #print NO[reorg[entry]]
        #print ''
        
        
        #print ''
        
        
        



4.0   Julio Jones ATL            WR

Julio is #1 in NFL history with 96.3 yards per game. With that being said, he is an increased injury risk over Brown and Beckham, plus he has been known to dis

21.0  T.Y. Hilton IND            WR

Did you know that Hilton led the league in receiving yards last year? He finished with 81 more yards than Odell Beckham Jr., though he scored just six touchdowns, which is the reason he's further down draft boards. It's now four straight 1,000-yard seasons for Hilton and Andrew Luck should finally be healthy this season.

28.0  Lamar Miller HOU           RB

Despite getting all the work he possibly could last year, Miller disappointed finishing with the sixth-most carries, but just the 17th most fantasy points among running backs. He misses two games, but the workload was there. The Texans spent a third-round pick to acquire D'Onta Foreman, which adds concern to that workload hanging around.

45.0  Allen Robinson JAC         WR

Betting on Robinson's tale

In [125]:
r  = requests.get('https://www.fantasypros.com/nfl/players/tom-brady.php')

# standard league
#https://www.fantasypros.com/nfl/rankings/consensus-cheatsheets.php

# [u'Rank\r\n\r\n', u'Player (Team)', u'Pos', u'Bye', \
#         0              1              2       3
#     u'Best', u'Worst', u'Avg', u'Std Dev', u'ADP', u'vs. ADP', u'Notes']
#        4          5      6        7           8         9        10


data = r.text

soup = BeautifulSoup(data)

tables = soup.findAll('table')

for indx,table in enumerate(tables): 
    
    print indx,[th.get_text() for th in table.find("tr").find_all("th")]
    
    #if [th.get_text() for th in table.find("tr").find_all("th")]==[u'Rushing Stats']:
        
    #if [th.get_text() for th in table.find("tr").find_all("th")]==[u'Receiving Stats']:
        
    #if [th.get_text() for th in table.find("tr").find_all("th")]==[u'Passing Stats']:



        #print [th.get_text() for th in table.find("tr").find_all("th")]
        
    for row in table.find_all("tr")[1:]:
            sav = [td.get_text() for td in row.find_all("td")]
            
            print sav
   

# receiving
# SEASON	TEAM	GAMES	REC	TGT	YDS	Y/R	LG	TD

# rushing 
# SEASON	TEAM	GAMES	ATT	YDS	Y/A	LG	TD	FUM	FUML

0 []
1 [u' ', u'ECR', u'Best', u'Worst', u'# of Experts']
[u'Draft', u'QB #2', u'QB #1', u'QB #6', u'104']
[u'Dynasty ', u'QB #10', u'QB #6', u'QB #15', u'19']
2 [u' ', u'Consensus', u'CBS', u'Fantrax', u'NFL', u'DW', u'FFC']
[u'Overall', u'#18', u'#16', u'#23', u'#23', u'#27', u'#30']
[u'Quarterbacks', u'#2', u'#2', u'#2', u'#2', u'#2', u'#2']
3 [u'Pass Cmp', u'Pass Att', u'Pass Yds', u'Pass Tds', u'Pass Ints', u'Rush Att', u'Rush Yds', u'Rush Tds', u'Fumbles', u'Points']
[u'390.3', u'597.2', u'4686.2', u'35.3', u'8.1', u'35.1', u'62.3', u'1.4', u'2.9', u'321.3']
4 [u' ', u'Pass Yds', u'Pass Tds', u'Rush Yds', u'Rush Tds', u'Points']
[u'Last Week', u'276', u'3', u'6', u'0', u'25.6']
[u'Last 3 Weeks', u'226', u'2', u'1.7', u'0', u'17.9']
[u'Season to Date', u'296.2', u'2.3', u'5.3', u'0', u'21.5']
5 [u' ', u'OPP', u'Pass Yds', u'Pass Tds', u'Rush Yds', u'Rush Tds', u'Points']
[u'Week 17', u'@ MIA', u'276', u'3', u'6', u'0', u'25.6']
[u'Week 16', u'vs. NYJ', u'214', u'3', u'0', u'0', u'